In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AdapterConfig, AdapterType

# Load the pretrained LLaMA 2 model
model_name = "meta-llama/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add an adapter
adapter_name = "sentiment-task"
model.add_adapter(adapter_name, AdapterType.text_task, config="pfeiffer")

# Activate the adapter
model.train_adapter(adapter_name)


In [ ]:
from transformers import TrainingArguments, Trainer
from datasets import load_dataset

# Load a dataset
dataset = load_dataset("imdb")
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./llama2-with-adapter",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    num_train_epochs=3
)

# Create a trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

# Train the adapter
trainer.train()

# Save the model with the trained adapter
model.save_adapter("./llama2-with-adapter", adapter_name)
